# Requirements

For Assignment 3, you will perform a short study to *interpret* a cultural pattern by articulating computational methods and social science theory. Your submission should be in a Jupyter notebook format and address the following prompts.

1. (3 points) Using computational methods, can you identify a popular song "formula" (or formulas) for the songs (or a subset of the songs) in the Billboard Hot 100 ranking data provided in music_df.csv (a la the discussion in Frith 1996: 160-161)? Make your case using at least one plot and a short (100+ word) description of your findings. Consider the lyrics as well as the various statistics about the songs included in music_df.csv. Details about these statistics can be found on the original Github data source descriptions for music_df.csv (from the people who compiled the data). More detailed information on the different Spotify music variables is available on the Spotify Developers website.

2. Interpret what drives this formula (or non-formulaic heterogeneity). Draw on social science theory (from this class and/or others) to make your interpretation (for instance, theories cited in the Frith 1996 and Du Gay et al. 2013 readings might be useful). Cite at least 3 peer-reviewed sources in order to make your case.

    - (3 points) What legisign relationship(s) should exist if your social science theory explains the observed pattern? How can you computationally test if this legisign relationship exists? Explain in 100+ words.
    - (5 points) Test the above legisign relationship(s) in your Jupyter notebook using the music_df.csv dataset and any additional information you may need to perform your test(s) (Note: it is optional to bring in external data). Produce at least one plot that helps you interpret the results of your test(s).
    - (4 points) Explain in words (300+) how the results of your analysis inform the interpretation of your identified formula/non-formulaic heterogeneity. What do you conclude?

# Identify a "Formula"

## Import and join datasets

## Sentiment analysis of the Hip-hop genre

### Positive, neutral, and negative scores

### Sentiment analysis of lyrics

# Interpret the "Formula"

## Legisign relationships

## Test the legisign relationship

## Explanations